In [109]:
# reload
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [111]:
import os
os.chdir('/rds/general/user/yz9023/home/LLA')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore, rankdata
import torch
import torch.nn as nn
import torch.optim as optim
from data_process import *
from training import *
from model import *
import xgboost as xgb
from utilis import *
from sklearn.model_selection import train_test_split

In [112]:
set_seed(42)

In [79]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# **Data Preparation**

In [6]:
MIMICtable = pd.read_csv('dataset/MIMICtable.csv')

In [7]:
num_epoch = 50  # 训练循环次数
gamma = 0.99
beat1 = 0
beat2 = 0.6
beta3 = 0.3
ncv = 5  # nr of crossvalidation runs (each is 80% training / 20% test)交叉验证运行的Nr(每次为80%训练/ 20%测试)
nra = 5
lr = 1e-5
reward_value = 24
beta = [beat1, beat2, beta3]
icustayidlist = MIMICtable['icustayid']
icuuniqueids = np.unique(icustayidlist)  # list of unique icustayids from MIMIC唯一的icustayid列表
reformat5 = MIMICtable.values.copy()
print('####  生成状态  ####')

# -----------------------筛选后的特征=37个--------------------------------
colnorm = ['SOFA', 'age', 'Weight_kg', 'GCS', 'HR', 'SysBP', 'MeanBP', 'DiaBP', 'RR', 'Temp_C',
           'Sodium', 'Chloride', 'Glucose', 'Calcium', 'Hb', 'WBC_count', 'Platelets_count',
           'PTT', 'PT', 'Arterial_pH', 'paO2', 'paCO2', 'HCO3', 'Arterial_lactate', 'Shock_Index',
           'PaO2_FiO2', 'cumulated_balance', 'CO2_mEqL', 'Ionised_Ca']
##8个指标
collog = ['SpO2', 'BUN', 'Creatinine', 'SGOT', 'Total_bili', 'INR', 'input_total', 'output_total']

colnorm = np.where(np.isin(MIMICtable.columns, colnorm))[0]
collog = np.where(np.isin(MIMICtable.columns, collog))[0]

####  生成状态  ####


In [8]:
train = np.load('dataset/train.npy')
validat = np.load('dataset/validation.npy')
test = np.load('dataset/test.npy')

In [9]:
scaleMIMIC = np.concatenate([zscore(reformat5[:, colnorm], ddof=1),
                              zscore(np.log(0.1 + reformat5[:, collog]), ddof=1)], axis=1)

Xvalidat = scaleMIMIC[validat, :]
blocsvalidat = reformat5[validat, 0]
ptidvalidat = reformat5[validat, 1]

Xtrain = scaleMIMIC[train, :]
Xtest = scaleMIMIC[test, :]
blocstrain = reformat5[train, 0]  # 序列号
bloctest = reformat5[test, 0]
ptidtrain = reformat5[train, 1]  # 患者编号
ptidtest = reformat5[test, 1]

# *************************
RNNstate = Xtrain  # ***

print('####  生成动作  ####')
nact = nra * nra  # 5*5=25
iol = MIMICtable.columns.get_loc('input_4hourly')  # 输入的列
vcl = MIMICtable.columns.get_loc('max_dose_vaso')  # 最大使用加压药量的列

a = reformat5[:, iol].copy()  # IV fluid  静脉输液复苏
a = rankdata(a[a > 0]) / a[a > 0].shape[0]  # excludes zero fluid (will be action 1)不包括零液体 将是动作1
iof = np.floor((a + 0.2499999999) * 4)  # converts iv volume in 4 actions 在4个动作中转换静脉输液量
a = reformat5[:, iol].copy()
a = np.where(a > 0)[0]  # location of non-zero fluid in big matrix
io = np.ones((reformat5.shape[0], 1))  # array of ones, by default
io[a] = (iof + 1).reshape(-1, 1)  # where more than zero fluid given: save actual action
io = io.ravel()  # 两者的本质都是想把多维的数组降为1维  注射有5个动作，已经通过秩进行判断

vc = reformat5[:, vcl].copy()
vcr = rankdata(vc[vc != 0]) / vc[vc != 0].size
vcr = np.floor((vcr + 0.249999999999) * 4)  # converts to 4 bins
vcr[vcr == 0] = 1
vc[vc != 0] = vcr + 1
vc[vc == 0] = 1

ma1 = np.array(
    [np.median(reformat5[io == 1, iol]), np.median(reformat5[io == 2, iol]), np.median(reformat5[io == 3, iol]),
        np.median(reformat5[io == 4, iol]), np.median(reformat5[io == 5, iol])])  # median dose of drug in all bins
ma2 = np.array(
    [np.median(reformat5[vc == 1, vcl]), np.median(reformat5[vc == 2, vcl]), np.median(reformat5[vc == 3, vcl]),
        np.median(reformat5[vc == 4, vcl]), np.median(reformat5[vc == 5, vcl])])

med = np.concatenate([io.reshape(-1, 1), vc.reshape(-1, 1)], axis=1)
uniqueValues, actionbloc = np.unique(med, axis=0, return_inverse=True)

actionbloctrain = actionbloc[train]  # ***
actionblocvalidat = actionbloc[validat]  # ***
actionbloctest = actionbloc[test]

ma2Values = ma2[uniqueValues[:, 1].astype('int64') - 1].reshape(-1, 1)
ma1Values = ma1[uniqueValues[:, 0].astype('int64') - 1].reshape(-1, 1)
uniqueValuesdose = np.concatenate([ma2Values, ma1Values], axis=1)  # median dose of each bin for all 25 actions
print('####  生成动作完成  ####')

####  生成动作  ####
####  生成动作完成  ####


In [10]:
outcome = 9
Y90 = reformat5[train, outcome]
r = np.array([reward_value, -reward_value]).reshape(1, -1)
r2 = r * (2 * (1 - Y90.reshape(-1, 1)) - 1)

# -----奖励函数准备-----------------------------
SOFA = reformat5[train, 57]  # ***
R3 = r2[:, 0]
R4 = (R3 + reward_value) / (2 * reward_value)
c = 0
bloc_max = max(blocstrain)

In [11]:
print(RNNstate.shape[0])
print('#### 生成轨迹 ####')

statesize = RNNstate.shape[1]
states = np.zeros((int(RNNstate.shape[0] * 1.2), statesize))
actions = np.zeros((int(RNNstate.shape[0] * 1.2), 1), dtype=int)
next_actions = np.zeros((int(RNNstate.shape[0] * 1.2), 1), dtype=int)
rewards = np.zeros((int(RNNstate.shape[0] * 1.2), 1))
next_states = np.zeros((int(RNNstate.shape[0] * 1.2), statesize))
done_flags = np.zeros((int(RNNstate.shape[0] * 1.2), 1))
bloc_num = np.zeros((int(RNNstate.shape[0] * 1.2), 1))
blocnum1 = 1

c = 0
bloc_num_reward = 0

for i in range(RNNstate.shape[0] - 1):
    states[c] = RNNstate[i, :]
    actions[c] = actionbloctrain[i]
    bloc_num[c] = blocnum1

    if blocstrain[i] != blocstrain[i + 1]:
        next_states1 = np.zeros(statesize)
        next_actions1 = -1
        done_flags1 = 1
        reward1 = -beat1 * SOFA[i] + R3[i]
        blocnum1 += 1
        bloc_num_reward = 0
    else:
        next_states1 = RNNstate[i + 1, :]
        next_actions1 = actionbloctrain[i + 1]
        done_flags1 = 0
        reward1 = -beat2 * (SOFA[i + 1] - SOFA[i])
        bloc_num_reward += 1

    next_states[c] = next_states1
    next_actions[c] = next_actions1
    rewards[c] = reward1
    done_flags[c] = done_flags1
    c += 1

# 处理最后一行
states[c] = RNNstate[-1, :]
actions[c] = actionbloctrain[-1]
bloc_num[c] = blocnum1
next_states1 = np.zeros(statesize)
next_actions1 = -1
done_flags1 = 1
reward1 = -beat1 * SOFA[-1] + R3[-1]
next_states[c] = next_states1
next_actions[c] = next_actions1
rewards[c] = reward1
done_flags[c] = done_flags1
c += 1

# 切片到实际大小
states = states[:c, :]
next_states = next_states[:c, :]
actions = actions[:c, :]
next_actions = next_actions[:c, :]
rewards = rewards[:c, :]
done_flags = done_flags[:c, :]
bloc_num = bloc_num[:c, :]

# 去掉不必要的维度
bloc_num = np.squeeze(bloc_num)
actions = np.squeeze(actions)
rewards = np.squeeze(rewards)
done_flags = np.squeeze(done_flags)

# 转换为torch张量
state = torch.FloatTensor(states).to(device)
next_state = torch.FloatTensor(next_states).to(device)
action = torch.LongTensor(actions).to(device)
next_action = torch.LongTensor(next_actions).to(device)
reward = torch.FloatTensor(rewards).to(device)
done = torch.FloatTensor(done_flags).to(device)
SOFAS = torch.LongTensor(SOFA).to(device)
batchs = (state, next_state, action, next_action, reward, done, bloc_num, SOFAS)

199312
#### 生成轨迹 ####


# **Separate action**

In [12]:
inv_action_map = {}
count = 0
for i in range(5):
    for j in range(5):
        inv_action_map[count] = [i, j]
        count += 1

In [13]:
actions_tuple = [None for i in range(len(actions))]
for i in range(len(actions)):
    actions_tuple[i] = inv_action_map[actions[i]]
actions_tuple = np.array(actions_tuple)

In [14]:
actions_iv = actions_tuple[:, 0]
actions_vaso = actions_tuple[:, 1]
print(actions_iv[:5])
print(actions_vaso[:5])

[2 2 2 2 2]
[0 0 0 0 0]


In [15]:
# check the zero ratio of the actions
print(np.sum(actions_iv == 0) / len(actions_iv))
print(np.sum(actions_vaso == 0) / len(actions_vaso))

0.28191478686682186
0.8624769206068877


In [16]:
len(actions_iv)

199312

In [17]:
# append action_iv and action_vaso as another two feature
states_with_action = np.concatenate([states, actions_iv.reshape(-1, 1), actions_vaso.reshape(-1, 1)], axis=1)

In [18]:
states_with_action.shape

(199312, 39)

In [19]:
blocstrain[:20]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  9.,  1.,  2.,  3.,  4.,  5.,  6.,
        7.,  8.,  9., 10., 11., 12.,  1.])

# **LSTM + Attention**

In [20]:
# 使用修改后的函数
states_padded, actions, lengths, action_nonflat = process_data(states_with_action, blocstrain, max_length=15)
pretrain_state_train, pretrain_state_valid, pretrain_len_train, pretrain_len_valid = train_test_split(states_padded, lengths, test_size=0.2, random_state=42)
pretrain_train_dataset = torch.utils.data.TensorDataset(pretrain_state_train, pretrain_len_train)
pretrain_valid_dataset = torch.utils.data.TensorDataset(pretrain_state_valid, pretrain_len_valid)
pretrain_train_loader = torch.utils.data.DataLoader(pretrain_train_dataset, batch_size=64, shuffle=True)
pretrain_valid_loader = torch.utils.data.DataLoader(pretrain_valid_dataset, batch_size=64, shuffle=False)

In [68]:
from importlib import reload
import model
reload(model)

pretrain_model = model.LSTMWithLocalAttention(
    input_dim=37,
    hidden_dim=128,
    attention_dim=64
).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(pretrain_model.parameters(), lr=1e-3)

# early stop
patience = 5
min_valid_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(30):
    pretrain_model.train()
    for i, (batch, leng) in enumerate(pretrain_train_loader):
        batch = batch.to(device)
        leng = leng.to(device)
        optimizer.zero_grad()
        pred = pretrain_model(batch, leng)
        loss = loss_fn(pred, batch)
        loss.backward()
        optimizer.step()
    print(f"Train Epoch {epoch}, Loss: {loss.item()}")

    pretrain_model.eval()
    valid_loss = 0
    with torch.no_grad():
        for i, (batch, leng) in enumerate(pretrain_valid_loader):
            batch = batch.to(device)
            leng = leng.to(device)
            pred = pretrain_model(batch, leng)
            loss = loss_fn(pred, batch)
            valid_loss += loss.item()
        valid_loss = valid_loss / len(pretrain_valid_loader)
        print(f"Valid Epoch {epoch}, Loss: {valid_loss}")

        if valid_loss < min_valid_loss:
            min_valid_loss = valid_loss
            best_model_state = pretrain_model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping triggered after epoch {epoch}")
            pretrain_model.load_state_dict(best_model_state)
            break

Train Epoch 0, Loss: 0.05656508356332779
Valid Epoch 0, Loss: 0.14241451094858348
Train Epoch 1, Loss: 0.022043762728571892
Valid Epoch 1, Loss: 0.10788669130609681
Train Epoch 2, Loss: 8.929070472717285
Valid Epoch 2, Loss: 0.09618235134985298
Train Epoch 3, Loss: 0.013477431610226631
Valid Epoch 3, Loss: 0.08955413873385017
Train Epoch 4, Loss: 0.00950117502361536
Valid Epoch 4, Loss: 0.0849690636775146
Train Epoch 5, Loss: 0.005978170316666365
Valid Epoch 5, Loss: 0.08180166207603179
Train Epoch 6, Loss: 0.0034354485105723143
Valid Epoch 6, Loss: 0.078759555821307
Train Epoch 7, Loss: 0.0066359988413751125
Valid Epoch 7, Loss: 0.07636309600881457
Train Epoch 8, Loss: 0.0031645644921809435
Valid Epoch 8, Loss: 0.07477074165459878
Train Epoch 9, Loss: 0.002218780340626836
Valid Epoch 9, Loss: 0.07332414088887163
Train Epoch 10, Loss: 0.004086521919816732
Valid Epoch 10, Loss: 0.07206040551439703
Train Epoch 11, Loss: 0.0019704382866621017
Valid Epoch 11, Loss: 0.07049365057173418
Trai

In [71]:
output = pretrain_model.forward_embedding(states_padded.to(device), lengths)


LSTM output shape: torch.Size([15218, 15, 128])


In [78]:
output.shape

torch.Size([180785, 128])

In [ ]:
from model import LSTMWithLocalAttention

def train_epoch(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for batch, lengths in train_loader:
        batch = batch.to(device)
        lengths = lengths.to(device)

        optimizer.zero_grad()
        pred = model(batch, lengths)
        loss = loss_fn(pred, batch)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(train_loader)

def validate(model, valid_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch, lengths in valid_loader:
            batch = batch.to(device)
            lengths = lengths.to(device)
            pred = model(batch, lengths)
            loss = loss_fn(pred, batch)
            total_loss += loss.item()
    return total_loss / len(valid_loader)

# Training configuration
config = {
    'input_dim': 37,
    'hidden_dim': 128,
    'attention_dim': 64,
    'dropout': 0.1,
    'lr': 1e-3,
    'patience': 5,
    'epochs': 30,
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}


lstm_attention_pretrain = LSTMWithLocalAttention(
    input_dim=config['input_dim'],
    hidden_dim=config['hidden_dim'],
    attention_dim=config['attention_dim'],
    dropout=config['dropout']
).to(config['device'])

loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(lstm_attention_pretrain.parameters(), lr=config['lr'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2, verbose=True
)

# Training loop with early stopping
min_valid_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(config['epochs']):
    valid_loss = validate(lstm_attention_pretrain, pretrain_valid_loader, loss_fn, config['device'])
    train_loss = train_epoch(lstm_attention_pretrain, pretrain_train_loader, optimizer, loss_fn, config['device'])

    print(f"Epoch {epoch}: Train Loss = {train_loss:.6f}, Valid Loss = {valid_loss:.6f}")

    # Learning rate scheduling
    scheduler.step(valid_loss)

    # Early stopping check
    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        best_model_state = lstm_attention_pretrain.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= config['patience']:
        print(f"Early stopping triggered after epoch {epoch}")
        lstm_attention_pretrain.load_state_dict(best_model_state)
        break

TypeError: __init__() got an unexpected keyword argument 'dropout'

In [22]:
output = lstm_attention_pretrain.forward_embedding(states_padded.to(device), lengths)

In [23]:
states_with_action.shape

(199312, 39)

In [24]:
output.shape

torch.Size([180785, 128])

In [25]:
actions.shape

torch.Size([180785, 2])

In [113]:
actions.shape

torch.Size([180785, 2])

In [114]:
action_nonflat.shape

torch.Size([15218, 15, 2])

# **LSTM**

In [84]:
class LSTM_recon(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LSTM_recon, self).__init__()

        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, input_dim)

    def forward(self, x, lengths):
        lengths = lengths.cpu()
        packed_input = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hn, cn) = self.lstm(packed_input)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)

        # reconstruction
        pred = self.output_layer(output)

        return pred

    def forward_embedding(self, x, lengths):
        original_length = x.size(1)
        lengths = lengths.cpu()
        packed_input = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hn, cn) = self.lstm(packed_input)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True, total_length=original_length)

        return output

In [82]:
from sklearn.model_selection import train_test_split
pretrain_state_train, pretrain_state_valid, pretrain_len_train, pretrain_len_valid = train_test_split(states_padded, lengths, test_size=0.2, random_state=42)
pretrain_train_dataset = torch.utils.data.TensorDataset(pretrain_state_train, pretrain_len_train)
pretrain_valid_dataset = torch.utils.data.TensorDataset(pretrain_state_valid, pretrain_len_valid)
pretrain_train_loader = torch.utils.data.DataLoader(pretrain_train_dataset, batch_size=64, shuffle=True)
pretrain_valid_loader = torch.utils.data.DataLoader(pretrain_valid_dataset, batch_size=64, shuffle=False)

In [85]:
pretrain_model = LSTM_recon(
    input_dim=37,
    hidden_dim=128
).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(pretrain_model.parameters(), lr=1e-3)

# early stop
patience = 5
min_valid_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(30):
    pretrain_model.train()
    for i, (batch, leng) in enumerate(pretrain_train_loader):
        batch = batch.to(device)
        leng = leng.to(device)
        optimizer.zero_grad()
        pred = pretrain_model(batch, leng)
        loss = loss_fn(pred, batch)
        loss.backward()
        optimizer.step()
    print(f"Train Epoch {epoch}, Loss: {loss.item()}")

    pretrain_model.eval()
    valid_loss = 0
    with torch.no_grad():
        for i, (batch, leng) in enumerate(pretrain_valid_loader):
            batch = batch.to(device)
            leng = leng.to(device)
            pred = pretrain_model(batch, leng)
            loss = loss_fn(pred, batch)
            valid_loss += loss.item()
        valid_loss = valid_loss / len(pretrain_valid_loader)
        print(f"Valid Epoch {epoch}, Loss: {valid_loss}")

        if valid_loss < min_valid_loss:
            min_valid_loss = valid_loss
            best_model_state = pretrain_model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping triggered after epoch {epoch}")
            pretrain_model.load_state_dict(best_model_state)
            break


Train Epoch 0, Loss: 0.060489632189273834
Valid Epoch 0, Loss: 0.1425508001508812
Train Epoch 1, Loss: 0.1725250780582428
Valid Epoch 1, Loss: 0.10802822300077726
Train Epoch 2, Loss: 0.009149150922894478
Valid Epoch 2, Loss: 0.09640381308660533
Train Epoch 3, Loss: 0.010798722505569458
Valid Epoch 3, Loss: 0.08945262274937704
Train Epoch 4, Loss: 0.013311183080077171
Valid Epoch 4, Loss: 0.08460557652870193
Train Epoch 5, Loss: 0.005889377556741238
Valid Epoch 5, Loss: 0.08102950103542146
Train Epoch 6, Loss: 0.004196146037429571
Valid Epoch 6, Loss: 0.07828582953273629
Train Epoch 7, Loss: 0.00396663649007678
Valid Epoch 7, Loss: 0.07591384405774686
Train Epoch 8, Loss: 0.0030941099394112825
Valid Epoch 8, Loss: 0.0743363931348237
Train Epoch 9, Loss: 0.0028802717570215464
Valid Epoch 9, Loss: 0.07261041092957991
Train Epoch 10, Loss: 0.002026542089879513
Valid Epoch 10, Loss: 0.07129087577050086
Train Epoch 11, Loss: 0.002410000190138817
Valid Epoch 11, Loss: 0.07032224821159616
Tra

In [86]:
states_padded.shape

torch.Size([15218, 15, 37])

In [88]:
lstm_ouput = pretrain_model.forward_embedding(states_padded.to(device), lengths)

In [89]:
actions.shape

torch.Size([180785, 2])

In [90]:
lstm_ouput.shape

torch.Size([15218, 15, 128])

In [91]:
action_nonflat.shape

torch.Size([15218, 15, 2])

# **XGBoost**

In [ ]:
# 使用修改后的函数
states_padded, actions, lengths, action_nonflat = process_data(states_with_action, blocstrain, max_length=15)
pretrain_state_train, pretrain_state_valid, pretrain_len_train, pretrain_len_valid = train_test_split(states_padded, lengths, test_size=0.2, random_state=42)
pretrain_train_dataset = torch.utils.data.TensorDataset(pretrain_state_train, pretrain_len_train)
pretrain_valid_dataset = torch.utils.data.TensorDataset(pretrain_state_valid, pretrain_len_valid)
pretrain_train_loader = torch.utils.data.DataLoader(pretrain_train_dataset, batch_size=64, shuffle=True)
pretrain_valid_loader = torch.utils.data.DataLoader(pretrain_valid_dataset, batch_size=64, shuffle=False)

### Baseline

In [171]:
ori_feature = []
for i in range(len(lengths)):
    ori_feature.append(states_padded[i, :lengths[i], :].cpu())
ori_feature = torch.cat(ori_feature, dim=0)
X_train, X_test, y_train_iol, y_test_iol = prepare_xgboost_data(ori_feature, actions[:,0])


Data shapes:
X_train shape: (144628, 37)
X_test shape: (36157, 37)
y_train shape: (144628,)
y_test shape: (36157,)
Unique action values: [0. 1. 2. 3. 4.]


In [172]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=250,
    learning_rate=1,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=10,
)

In [173]:
xgb_model = train_xgboost_classifier(xgb_model, X_train, y_train_iol, X_test, y_test_iol)
evaluate_model(xgb_model, X_train, X_test, y_train_iol, y_test_iol)

Class 0: count = 41625.0, weight = 0.6692
Class 1: count = 25552.0, weight = 1.0902
Class 2: count = 24984.0, weight = 1.1149
Class 3: count = 25427.0, weight = 1.0955
Class 4: count = 27040.0, weight = 1.0302
[0]	validation_0-auc:0.72407	validation_1-auc:0.71490
[1]	validation_0-auc:0.74762	validation_1-auc:0.73547
[2]	validation_0-auc:0.75799	validation_1-auc:0.74286
[3]	validation_0-auc:0.76489	validation_1-auc:0.74714
[4]	validation_0-auc:0.77190	validation_1-auc:0.75195
[5]	validation_0-auc:0.77841	validation_1-auc:0.75642
[6]	validation_0-auc:0.78326	validation_1-auc:0.75793
[7]	validation_0-auc:0.78741	validation_1-auc:0.75915
[8]	validation_0-auc:0.79126	validation_1-auc:0.76091
[9]	validation_0-auc:0.79657	validation_1-auc:0.76327
[10]	validation_0-auc:0.79981	validation_1-auc:0.76483
[11]	validation_0-auc:0.80358	validation_1-auc:0.76629
[12]	validation_0-auc:0.80705	validation_1-auc:0.76792
[13]	validation_0-auc:0.81043	validation_1-auc:0.76898
[14]	validation_0-auc:0.81377	

### LSTM Pretraining

In [188]:
from importlib import reload
import model
reload(model)

<module 'model' from '/rds/general/user/yz9023/home/LLA/model.py'>

In [189]:
pretrain_model = model.LSTM_pretrain(
    input_dim=37,
    hidden_dim=512
).to(device)
initialize_weights(pretrain_model)

loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(pretrain_model.parameters(), lr=1e-3)

# early stop
patience = 5
min_valid_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(100):
    pretrain_model.eval()
    valid_loss = 0
    with torch.no_grad():
        for i, (batch, leng) in enumerate(pretrain_valid_loader):
            batch = batch.to(device)
            leng = leng.to(device)
            pred = pretrain_model(batch, leng)
            loss = loss_fn(pred, batch)
            valid_loss += loss.item()
        valid_loss = valid_loss / len(pretrain_valid_loader)
        print(f"Valid Epoch {epoch}, Loss: {valid_loss}")
        
    pretrain_model.train()
    total_loss = 0
    for i, (batch, leng) in enumerate(pretrain_train_loader):
        batch = batch.to(device)
        leng = leng.to(device)
        optimizer.zero_grad()
        pred = pretrain_model(batch, leng)
        loss = loss_fn(pred, batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Epoch {epoch}, Loss: {total_loss / len(pretrain_train_loader)}")

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        best_model_state = pretrain_model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping triggered after epoch {epoch}")
        pretrain_model.load_state_dict(best_model_state)
        break

Valid Epoch 0, Loss: 0.8516409918665886
Train Epoch 0, Loss: 0.14464779674070669
Valid Epoch 1, Loss: 0.08187818185736735
Train Epoch 1, Loss: 0.03107025631805833
Valid Epoch 2, Loss: 0.0706621435820125
Train Epoch 2, Loss: 0.023727760936684557
Valid Epoch 3, Loss: 0.07200330518147287
Train Epoch 3, Loss: 0.020077661904920847
Valid Epoch 4, Loss: 0.06340354312123964
Train Epoch 4, Loss: 0.01743371063643452
Valid Epoch 5, Loss: 0.06168923385606225
Train Epoch 5, Loss: 0.0155127216474865
Valid Epoch 6, Loss: 0.06099143423489295
Train Epoch 6, Loss: 0.014223922584364336
Valid Epoch 7, Loss: 0.0607281109575221
Train Epoch 7, Loss: 0.014660188038065463
Valid Epoch 8, Loss: 0.05911741368496829
Train Epoch 8, Loss: 0.013133424336666056
Valid Epoch 9, Loss: 0.05793330386222806
Train Epoch 9, Loss: 0.01185772533702825
Valid Epoch 10, Loss: 0.05823580093177346
Train Epoch 10, Loss: 0.010966873364779046
Valid Epoch 11, Loss: 0.05677498161821859
Train Epoch 11, Loss: 0.009444193327580019
Valid Epo

In [194]:
output = pretrain_model.forward_embedding(states_padded.to(device), lengths)
X_train, X_test, y_train_iol, y_test_iol = prepare_xgboost_data(output, actions[:,0])


Data shapes:
X_train shape: (144628, 512)
X_test shape: (36157, 512)
y_train shape: (144628,)
y_test shape: (36157,)
Unique action values: [0. 1. 2. 3. 4.]


In [196]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=250,
    learning_rate=1,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=10,
)
xgb_model = train_xgboost_classifier(xgb_model, X_train, y_train_iol, X_test, y_test_iol)
evaluate_model(xgb_model, X_train, X_test, y_train_iol, y_test_iol)

Class 0: count = 41625.0, weight = 0.6692
Class 1: count = 25552.0, weight = 1.0902
Class 2: count = 24984.0, weight = 1.1149
Class 3: count = 25427.0, weight = 1.0955
Class 4: count = 27040.0, weight = 1.0302
[0]	validation_0-auc:0.69460	validation_1-auc:0.67289
[1]	validation_0-auc:0.72141	validation_1-auc:0.69287
[2]	validation_0-auc:0.73641	validation_1-auc:0.70212
[3]	validation_0-auc:0.74946	validation_1-auc:0.70971
[4]	validation_0-auc:0.75960	validation_1-auc:0.71486
[5]	validation_0-auc:0.76852	validation_1-auc:0.71862
[6]	validation_0-auc:0.77632	validation_1-auc:0.72179
[7]	validation_0-auc:0.78356	validation_1-auc:0.72435
[8]	validation_0-auc:0.78968	validation_1-auc:0.72680
[9]	validation_0-auc:0.79611	validation_1-auc:0.72856
[10]	validation_0-auc:0.80131	validation_1-auc:0.73069
[11]	validation_0-auc:0.80703	validation_1-auc:0.73196
[12]	validation_0-auc:0.81263	validation_1-auc:0.73321
[13]	validation_0-auc:0.81726	validation_1-auc:0.73427
[14]	validation_0-auc:0.82201	

### LSTM WITHOUT Pretraining

In [175]:
pretrain_model = model.LSTM_pretrain(
    input_dim=37,
    hidden_dim=512,
).to(device)
initialize_weights(pretrain_model)

In [176]:
output = pretrain_model.forward_embedding(states_padded.to(device), lengths)
X_train, X_test, y_train_iol, y_test_iol = prepare_xgboost_data(output, actions[:,0])


Data shapes:
X_train shape: (144628, 512)
X_test shape: (36157, 512)
y_train shape: (144628,)
y_test shape: (36157,)
Unique action values: [0. 1. 2. 3. 4.]


In [177]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=250,
    learning_rate=1,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=10,
)

In [178]:
xgb_model = train_xgboost_classifier(xgb_model, X_train, y_train_iol, X_test, y_test_iol)
evaluate_model(xgb_model, X_train, X_test, y_train_iol, y_test_iol)

Class 0: count = 41625.0, weight = 0.6692
Class 1: count = 25552.0, weight = 1.0902
Class 2: count = 24984.0, weight = 1.1149
Class 3: count = 25427.0, weight = 1.0955
Class 4: count = 27040.0, weight = 1.0302
[0]	validation_0-auc:0.69011	validation_1-auc:0.66810
[1]	validation_0-auc:0.71628	validation_1-auc:0.68734
[2]	validation_0-auc:0.73245	validation_1-auc:0.69635
[3]	validation_0-auc:0.74465	validation_1-auc:0.70374
[4]	validation_0-auc:0.75535	validation_1-auc:0.70955
[5]	validation_0-auc:0.76568	validation_1-auc:0.71451
[6]	validation_0-auc:0.77322	validation_1-auc:0.71848
[7]	validation_0-auc:0.78053	validation_1-auc:0.72226
[8]	validation_0-auc:0.78703	validation_1-auc:0.72493
[9]	validation_0-auc:0.79329	validation_1-auc:0.72762
[10]	validation_0-auc:0.79954	validation_1-auc:0.73026
[11]	validation_0-auc:0.80516	validation_1-auc:0.73321
[12]	validation_0-auc:0.81046	validation_1-auc:0.73523
[13]	validation_0-auc:0.81591	validation_1-auc:0.73665
[14]	validation_0-auc:0.82155	

### LSTM_Attention Pretraining

In [143]:
from importlib import reload
import model
reload(model)

<module 'model' from '/rds/general/user/yz9023/home/LLA/model.py'>

In [149]:
pretrain_model = model.LSTMAttentionModel(
    input_dim=37,
    hidden_dim=512,
    attn_layers=1,
).to(device)
initialize_weights(pretrain_model)
print(pretrain_model)

loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(pretrain_model.parameters(), lr=1e-3)

# early stop
patience = 5
min_valid_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(100):
    pretrain_model.eval()
    valid_loss = 0
    with torch.no_grad():
        for i, (batch, leng) in enumerate(pretrain_valid_loader):
            batch = batch.to(device)
            leng = leng.to(device)
            pred = pretrain_model(batch, leng)
            loss = loss_fn(pred, batch)
            valid_loss += loss.item()
        valid_loss = valid_loss / len(pretrain_valid_loader)
        print(f"Valid Epoch {epoch}, Loss: {valid_loss}")
        
    pretrain_model.train()
    total_loss = 0
    for i, (batch, leng) in enumerate(pretrain_train_loader):
        batch = batch.to(device)
        leng = leng.to(device)
        optimizer.zero_grad()
        pred = pretrain_model(batch, leng)
        loss = loss_fn(pred, batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Epoch {epoch}, Loss: {total_loss / len(pretrain_train_loader)}")

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        best_model_state = pretrain_model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping triggered after epoch {epoch}")
        pretrain_model.load_state_dict(best_model_state)
        break

LSTMAttentionModel(
  (lstm): LSTM(37, 512, batch_first=True)
  (atten_layers): Sequential(
    (0): CausalAttention(
      (query_fc): Linear(in_features=512, out_features=512, bias=True)
      (key_fc): Linear(in_features=512, out_features=512, bias=True)
      (value_fc): Linear(in_features=512, out_features=512, bias=True)
      (softmax): Softmax(dim=-1)
    )
  )
  (prediction_head): Linear(in_features=512, out_features=37, bias=True)
)
Valid Epoch 0, Loss: 0.8522433613737425
Train Epoch 0, Loss: 0.10659781746878362
Valid Epoch 1, Loss: 0.061644367582630366
Train Epoch 1, Loss: 0.0231855582319083
Valid Epoch 2, Loss: 0.05600588759989478
Train Epoch 2, Loss: 0.012265184966125647
Valid Epoch 3, Loss: 0.05751318080486575
Train Epoch 3, Loss: 0.01450447874455786
Valid Epoch 4, Loss: 0.07681702957294571
Train Epoch 4, Loss: 0.042432176389277994
Valid Epoch 5, Loss: 0.0690287012742677
Train Epoch 5, Loss: 0.04346977354451546
Valid Epoch 6, Loss: 0.07704652791532378
Train Epoch 6, Loss:

In [150]:
pretrain_model.eval()
output = pretrain_model.forward_embedding(states_padded.to(device), lengths)
X_train, X_test, y_train_iol, y_test_iol = prepare_xgboost_data(output, actions[:,0])


Data shapes:
X_train shape: (144628, 512)
X_test shape: (36157, 512)
y_train shape: (144628,)
y_test shape: (36157,)
Unique action values: [0. 1. 2. 3. 4.]


In [151]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=250,
    learning_rate=1,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=10,
)

In [152]:
xgb_model = train_xgboost_classifier(xgb_model, X_train, y_train_iol, X_test, y_test_iol)
evaluate_model(xgb_model, X_train, X_test, y_train_iol, y_test_iol)

Class 0: count = 41625.0, weight = 0.6692
Class 1: count = 25552.0, weight = 1.0902
Class 2: count = 24984.0, weight = 1.1149
Class 3: count = 25427.0, weight = 1.0955
Class 4: count = 27040.0, weight = 1.0302
[0]	validation_0-auc:0.69383	validation_1-auc:0.67219
[1]	validation_0-auc:0.72201	validation_1-auc:0.69355
[2]	validation_0-auc:0.73788	validation_1-auc:0.70300
[3]	validation_0-auc:0.75012	validation_1-auc:0.70982
[4]	validation_0-auc:0.76012	validation_1-auc:0.71331
[5]	validation_0-auc:0.76871	validation_1-auc:0.71751
[6]	validation_0-auc:0.77564	validation_1-auc:0.72043
[7]	validation_0-auc:0.78224	validation_1-auc:0.72234
[8]	validation_0-auc:0.78896	validation_1-auc:0.72501
[9]	validation_0-auc:0.79492	validation_1-auc:0.72687
[10]	validation_0-auc:0.79991	validation_1-auc:0.72829
[11]	validation_0-auc:0.80404	validation_1-auc:0.72924
[12]	validation_0-auc:0.80870	validation_1-auc:0.72993
[13]	validation_0-auc:0.81359	validation_1-auc:0.73118
[14]	validation_0-auc:0.81751	

### GPT pretraining

---

In [197]:
from importlib import reload
import model
reload(model)

<module 'model' from '/rds/general/user/yz9023/home/LLA/model.py'>

In [198]:
input_dim = 37
hidden_dim = 512
num_layers = 1
num_heads = 1
dropout = 0.05

pretrain_model = model.GPTCausalAttentionModel(input_dim=input_dim, hidden_dim=hidden_dim,
                                num_layers=num_layers, num_heads=num_heads, dropout=dropout).to(device)
initialize_weights(pretrain_model)
loss_fn = nn.MSELoss()
optimizer = torch.optim.AdamW(pretrain_model.parameters(), lr=1e-3)

# early stop
patience = 10
min_valid_loss = float('inf')
patience_counter = 0
best_model_state = None

for epoch in range(100):
    pretrain_model.eval()
    valid_loss = 0
    with torch.no_grad():
        for i, (batch, leng) in enumerate(pretrain_valid_loader):
            batch = batch.to(device)
            pred = pretrain_model(batch)
            loss = loss_fn(pred, batch)
            valid_loss += loss.item()
        valid_loss = valid_loss / len(pretrain_valid_loader)
        print(f"Valid Epoch {epoch}, Loss: {valid_loss}")
        
    pretrain_model.train()
    total_loss = 0
    for i, (batch, leng) in enumerate(pretrain_train_loader):
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = pretrain_model(batch)
        loss = loss_fn(pred, batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Train Epoch {epoch}, Loss: {total_loss / len(pretrain_train_loader)}")

    if valid_loss < min_valid_loss:
        min_valid_loss = valid_loss
        best_model_state = pretrain_model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping triggered after epoch {epoch}")
        pretrain_model.load_state_dict(best_model_state)
        break

Valid Epoch 0, Loss: 2.7223497877518334
Train Epoch 0, Loss: 0.20193532688533447
Valid Epoch 1, Loss: 0.10584097281874467
Train Epoch 1, Loss: 0.0677739415922402
Valid Epoch 2, Loss: 0.1022250772997116
Train Epoch 2, Loss: 0.05055858034406501
Valid Epoch 3, Loss: 0.09215145835575338
Train Epoch 3, Loss: 0.04208117615704605
Valid Epoch 4, Loss: 0.087874471374865
Train Epoch 4, Loss: 0.032200103081977335
Valid Epoch 5, Loss: 0.07994613529202373
Train Epoch 5, Loss: 0.030916281349235335
Valid Epoch 6, Loss: 0.10394537798129022
Train Epoch 6, Loss: 0.03293654611651147
Valid Epoch 7, Loss: 0.07664201869435298
Train Epoch 7, Loss: 0.022717429326203558
Valid Epoch 8, Loss: 0.07268328492015523
Train Epoch 8, Loss: 0.13762648592956597
Valid Epoch 9, Loss: 0.4379807561635971
Train Epoch 9, Loss: 0.10616665976658385
Valid Epoch 10, Loss: 0.11453345472303529
Train Epoch 10, Loss: 0.0530680188621487
Valid Epoch 11, Loss: 0.08635234086735484
Train Epoch 11, Loss: 0.03360973806632439
Valid Epoch 12, 

In [199]:
pretrain_model.eval()
output = pretrain_model.forward_embedding(states_padded.to(device), lengths)
X_train, X_test, y_train_iol, y_test_iol = prepare_xgboost_data(output, actions[:,0])


Data shapes:
X_train shape: (144628, 512)
X_test shape: (36157, 512)
y_train shape: (144628,)
y_test shape: (36157,)
Unique action values: [0. 1. 2. 3. 4.]


In [200]:
xgb_model = xgb.XGBClassifier(
    objective='multi:softprob',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=250,
    learning_rate=1,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=10,
)
xgb_model = train_xgboost_classifier(xgb_model, X_train, y_train_iol, X_test, y_test_iol)
evaluate_model(xgb_model, X_train, X_test, y_train_iol, y_test_iol)

Class 0: count = 41625.0, weight = 0.6692
Class 1: count = 25552.0, weight = 1.0902
Class 2: count = 24984.0, weight = 1.1149
Class 3: count = 25427.0, weight = 1.0955
Class 4: count = 27040.0, weight = 1.0302
[0]	validation_0-auc:0.69128	validation_1-auc:0.67065
[1]	validation_0-auc:0.71679	validation_1-auc:0.68865
[2]	validation_0-auc:0.73275	validation_1-auc:0.69714
[3]	validation_0-auc:0.74533	validation_1-auc:0.70322
[4]	validation_0-auc:0.75627	validation_1-auc:0.70834
[5]	validation_0-auc:0.76587	validation_1-auc:0.71255
[6]	validation_0-auc:0.77340	validation_1-auc:0.71623
[7]	validation_0-auc:0.78086	validation_1-auc:0.71831
[8]	validation_0-auc:0.78642	validation_1-auc:0.72045
[9]	validation_0-auc:0.79226	validation_1-auc:0.72256
[10]	validation_0-auc:0.79808	validation_1-auc:0.72448
[11]	validation_0-auc:0.80341	validation_1-auc:0.72585
[12]	validation_0-auc:0.80837	validation_1-auc:0.72700
[13]	validation_0-auc:0.81310	validation_1-auc:0.72791
[14]	validation_0-auc:0.81732	

---

In [ ]:
results = evaluate_random_short_sequences(X_test, y_test_iol, xgb_model, n_samples = 500)

print("\nOverall Statistics:")
total_predictions = sum(len(r['predictions']) for r in results)
total_correct = sum(np.sum(r['true_labels'] == r['predictions']) for r in results)
total_confident = sum(np.sum(r['is_confident']) for r in results)

print(f"Total accuracy: {total_correct/total_predictions:.4f}")
print(f"Confident states ratio: {total_confident/total_predictions:.4f}")
print(f"Average entropy: {np.mean([np.mean(r['entropies']) for r in results]):.4f}")

流式输出内容被截断，只能显示最后 5000 行内容。
    2     |    2.0     |     2     |   0.8376   | 0.6442  | True
    3     |    0.0     |     0     |   0.9684   | 0.1789  | True
    4     |    4.0     |     2     |   0.4609   | 1.1800  | False
    5     |    4.0     |     3     |   0.5206   | 1.1289  | False

Probability distributions:
Time step | Class    0   | Class    1   | Class    2   | Class    3   | Class    4   | 
-----------------------------------------------------------------
    1     | 0.9811 | 0.0123 | 0.0007 | 0.0034 | 0.0025 | 
    2     | 0.0080 | 0.0452 | 0.8376 | 0.0591 | 0.0501 | 
    3     | 0.9684 | 0.0137 | 0.0019 | 0.0080 | 0.0079 | 
    4     | 0.0581 | 0.0598 | 0.4609 | 0.0369 | 0.3843 | 
    5     | 0.0175 | 0.0415 | 0.0926 | 0.5206 | 0.3278 | 


Sequence 252 (starting from index 14480):
Time step | True Label | Predicted | Confidence | Entropy | Confident?
-----------------------------------------------------------------
    1     |    4.0     |     4     |   0.8497   | 0.5927  

In [ ]:
results = evaluate_random_short_sequences(X_test, y_test_vaso, xgb_model_2,n_samples = 500)

print("\nOverall Statistics:")
total_predictions = sum(len(r['predictions']) for r in results)
total_correct = sum(np.sum(r['true_labels'] == r['predictions']) for r in results)
total_confident = sum(np.sum(r['is_confident']) for r in results)

print(f"Total accuracy: {total_correct/total_predictions:.4f}")
print(f"Confident states ratio: {total_confident/total_predictions:.4f}")
print(f"Average entropy: {np.mean([np.mean(r['entropies']) for r in results]):.4f}")

流式输出内容被截断，只能显示最后 5000 行内容。
    2     |    0.0     |     0     |   0.8354   | 0.6311  | True
    3     |    0.0     |     0     |   0.9841   | 0.0977  | True
    4     |    4.0     |     0     |   0.5995   | 0.9831  | False
    5     |    0.0     |     0     |   0.9998   | 0.0021  | True

Probability distributions:
Time step | Class    0   | Class    1   | Class    2   | Class    3   | Class    4   | 
-----------------------------------------------------------------
    1     | 0.9971 | 0.0006 | 0.0001 | 0.0007 | 0.0014 | 
    2     | 0.8354 | 0.0247 | 0.0386 | 0.0917 | 0.0097 | 
    3     | 0.9841 | 0.0086 | 0.0055 | 0.0009 | 0.0008 | 
    4     | 0.5995 | 0.0125 | 0.0244 | 0.0602 | 0.3034 | 
    5     | 0.9998 | 0.0001 | 0.0000 | 0.0000 | 0.0001 | 


Sequence 252 (starting from index 5500):
Time step | True Label | Predicted | Confidence | Entropy | Confident?
-----------------------------------------------------------------
    1     |    0.0     |     0     |   0.9985   | 0.0132  | 

## **LSTM**

In [92]:
def reshape_for_xgboost(data):
    samples = data.shape[0]
    return data.reshape(samples, -1)

In [110]:
lstm_ouput.shape

torch.Size([15218, 15, 128])

In [109]:
action_nonflat[:,0].shape

torch.Size([15218, 2])

In [123]:
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = prepare_xgboost_data(lstm_ouput, action_nonflat[:,0])
X_train_reshaped = reshape_for_xgboost(X_train_lstm)
X_test_reshaped = reshape_for_xgboost(X_test_lstm)


Data shapes:
X_train shape: (12174, 15, 128)
X_test shape: (3044, 15, 128)
y_train shape: (12174, 2)
y_test shape: (3044, 2)
Unique action values: [0. 1. 2. 3. 4.]


In [96]:
def train_xgboost_classifier(X_train, y_train, X_test, y_test):
    """
    训练XGBoost分类器并评估
    """
    y_train = y_train.astype(int)
    y_test = y_test.astype(int)
    sample_weights = calculate_class_weights(y_train)

    xgb_model = xgb.XGBClassifier(
        objective='multi:softprob',
        eval_metric='auc',
        use_label_encoder=False,
        n_estimators=200,
        learning_rate=0.1,
        max_depth=6,
        min_child_weight=5,
        subsample=0.8,
        colsample_bytree=0.8,
        n_jobs=8,
    )

    xgb_model.fit(
        X_train,
        y_train,
        sample_weight=sample_weights,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=True
    )

    return xgb_model
xgb_model_lstm = train_xgboost_classifier(X_train_reshaped, y_train_lstm[:,0], X_test_reshaped, y_test_lstm[:,0])
evaluate_model(xgb_model_lstm, X_train_reshaped, X_test_reshaped, y_train_lstm[:,0], y_test_lstm[:,0])

Class 0: count = 7543.0, weight = 0.1847
Class 1: count = 1116.0, weight = 1.2481
Class 2: count = 1172.0, weight = 1.1884
Class 3: count = 1148.0, weight = 1.2133
Class 4: count = 1195.0, weight = 1.1656
[0]	validation_0-auc:0.86273	validation_1-auc:0.78186
[1]	validation_0-auc:0.90009	validation_1-auc:0.81211
[2]	validation_0-auc:0.91567	validation_1-auc:0.82536
[3]	validation_0-auc:0.92506	validation_1-auc:0.83526
[4]	validation_0-auc:0.93133	validation_1-auc:0.83978
[5]	validation_0-auc:0.93770	validation_1-auc:0.84319
[6]	validation_0-auc:0.94197	validation_1-auc:0.84766
[7]	validation_0-auc:0.94432	validation_1-auc:0.85078
[8]	validation_0-auc:0.94709	validation_1-auc:0.85222
[9]	validation_0-auc:0.94970	validation_1-auc:0.85472
[10]	validation_0-auc:0.95236	validation_1-auc:0.85700
[11]	validation_0-auc:0.95393	validation_1-auc:0.85829
[12]	validation_0-auc:0.95588	validation_1-auc:0.86048
[13]	validation_0-auc:0.95741	validation_1-auc:0.86158
[14]	validation_0-auc:0.95889	valid

In [105]:
lstm_ouput.shape

torch.Size([15218, 15, 128])

In [106]:
actions[:,0].shape

torch.Size([180785])

In [115]:
all_lstm_outputs = []
for i, lstm_o in enumerate(lstm_ouput):
    # 只取实际长度的action数据
    actual_length = lengths[i]
    all_lstm_outputs.append(lstm_o[:actual_length])
all_lstm_outputs = torch.cat(all_lstm_outputs, dim=0)

In [127]:
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = prepare_xgboost_data(all_lstm_outputs, actions[:,0])
X_train_reshaped = reshape_for_xgboost(X_train_lstm)
X_test_reshaped = reshape_for_xgboost(X_test_lstm)

def train_xgboost_classifier(X_train, y_train, X_test, y_test):
    """
    训练XGBoost分类器并评估
    """
    y_train = y_train.astype(int)
    y_test = y_test.astype(int)
    sample_weights = calculate_class_weights(y_train)

    xgb_model = xgb.XGBClassifier(
        objective='multi:softprob',
        eval_metric='auc',
        use_label_encoder=False,
        n_estimators=200,
        learning_rate=0.1,
        max_depth=6,
        min_child_weight=5,
        subsample=0.8,
        colsample_bytree=0.8,
        n_jobs=8,
    )

    xgb_model.fit(
        X_train,
        y_train,
        sample_weight=sample_weights,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=True
    )

    return xgb_model
xgb_model_lstm = train_xgboost_classifier(X_train_reshaped, y_train_lstm, X_test_reshaped, y_test_lstm)
evaluate_model(xgb_model_lstm, X_train_reshaped, X_test_reshaped, y_train_lstm, y_test_lstm)


Data shapes:
X_train shape: (144628, 128)
X_test shape: (36157, 128)
y_train shape: (144628,)
y_test shape: (36157,)
Unique action values: [0. 1. 2. 3. 4.]
Class 0: count = 41625.0, weight = 0.6692
Class 1: count = 25552.0, weight = 1.0902
Class 2: count = 24984.0, weight = 1.1149
Class 3: count = 25427.0, weight = 1.0955
Class 4: count = 27040.0, weight = 1.0302
[0]	validation_0-auc:0.68014	validation_1-auc:0.65880
[1]	validation_0-auc:0.69865	validation_1-auc:0.67516
[2]	validation_0-auc:0.70953	validation_1-auc:0.68352
[3]	validation_0-auc:0.71655	validation_1-auc:0.68868
[4]	validation_0-auc:0.72006	validation_1-auc:0.69199
[5]	validation_0-auc:0.72342	validation_1-auc:0.69389
[6]	validation_0-auc:0.72556	validation_1-auc:0.69583
[7]	validation_0-auc:0.72772	validation_1-auc:0.69728
[8]	validation_0-auc:0.72966	validation_1-auc:0.69861
[9]	validation_0-auc:0.73177	validation_1-auc:0.69970
[10]	validation_0-auc:0.73373	validation_1-auc:0.70080
[11]	validation_0-auc:0.73550	validati

In [ ]:
results = evaluate_random_short_sequences(X_test_reshaped, y_test_lstm[:,0], xgb_model_lstm, n_samples = 500)

print("\nOverall Statistics:")
total_predictions = sum(len(r['predictions']) for r in results)
total_correct = sum(np.sum(r['true_labels'] == r['predictions']) for r in results)
total_confident = sum(np.sum(r['is_confident']) for r in results)

print(f"Total accuracy: {total_correct/total_predictions:.4f}")
print(f"Confident states ratio: {total_confident/total_predictions:.4f}")
print(f"Average entropy: {np.mean([np.mean(r['entropies']) for r in results]):.4f}")

流式输出内容被截断，只能显示最后 5000 行内容。
    2     |    1.0     |     1     |   0.8267   | 0.6394  | True
    3     |    0.0     |     0     |   0.9886   | 0.0714  | True
    4     |    2.0     |     0     |   0.8899   | 0.4754  | True
    5     |    2.0     |     3     |   0.3686   | 1.4469  | False

Probability distributions:
Time step | Class    0   | Class    1   | Class    2   | Class    3   | Class    4   | 
-----------------------------------------------------------------
    1     | 0.9940 | 0.0028 | 0.0003 | 0.0014 | 0.0015 | 
    2     | 0.0872 | 0.8267 | 0.0633 | 0.0200 | 0.0027 | 
    3     | 0.9886 | 0.0013 | 0.0005 | 0.0009 | 0.0087 | 
    4     | 0.8899 | 0.0582 | 0.0279 | 0.0110 | 0.0131 | 
    5     | 0.0966 | 0.0614 | 0.2282 | 0.3686 | 0.2452 | 


Sequence 252 (starting from index 890):
Time step | True Label | Predicted | Confidence | Entropy | Confident?
-----------------------------------------------------------------
    1     |    0.0     |     0     |   0.9736   | 0.1262  | T